In [1]:
# Notebook to calculate some example numerical values quoted in the paper

import climepi  # noqa
import holoviews as hv
from climepi import climdata, epimod

import opts

# Get options
options = opts.get_opts()
year_range = options["year_range"]
location = options["location_default"]
epi_model_name = options["epi_model_name_default"]
ensemble_stats_kwargs = options["ensemble_stats_kwargs"]
plot_opts_clim = options["plot_opts_clim"]
plot_opts_epi = options["plot_opts_epi"]
data_base_dir = options["data_base_dir"]

colors = hv.Cycle().values

In [2]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset("isimip_cities_daily", base_dir=data_base_dir)
    .sel(location=location)
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
)
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]
# Get and run epidemiological model
epi_model = epimod.get_example_model(epi_model_name)
ds_suitability = ds_clim.climepi.run_epi_model(epi_model)
ds_days_suitable = ds_suitability.climepi.yearly_portion_suitable()
ds_days_suitable["time"] = ds_days_suitable["time.year"]

In [3]:
# Example proportions of uncertainty due to internal variability in a single year

year_example = 2080


def _get_icv_uncertainty_proportion(ds, time=None, data_var=None):
    da_decomp = ds.climepi.uncertainty_interval_decomposition(
        data_var, **ensemble_stats_kwargs
    )[data_var].sel(time=time)
    internal_width = (
        da_decomp.sel(level="internal_upper") - da_decomp.sel(level="internal_lower")
    ).values
    total_width = (
        da_decomp.sel(level="scenario_upper") - da_decomp.sel(level="scenario_lower")
    ).values
    return internal_width / total_width


prop_temp = _get_icv_uncertainty_proportion(
    ds_temp_yearly, time=year_example, data_var="temperature"
)
print(
    f"Proportion of uncertainty in temperature in {year_example} "
    f"due to internal variability: {prop_temp * 100:.2f}%"
)
prop_epi = _get_icv_uncertainty_proportion(
    ds_days_suitable, time=year_example, data_var="portion_suitable"
)
print(
    f"Proportion of uncertainty in months suitable in {year_example} "
    f"due to internal variability: {prop_epi * 100:.2f}%"
)

Proportion of uncertainty in temperature in 2080 due to internal variability: 38.47%
Proportion of uncertainty in months suitable in 2080 due to internal variability: 47.56%


In [4]:
# Example variance components due to ICV for different scenarios/models in a single year


def _get_ensemble_var_vals(ds, data_var=None):
    ensemble_var = (
        ds.climepi.ensemble_stats(data_var, **ensemble_stats_kwargs)[data_var]
        .isel(time=0)
        .sel(stat="var")
        .compute()
    )
    var_str = f"Mean: {ensemble_var.mean().values}\n"
    for scenario in ensemble_var.scenario.values:
        ensemble_var_scenario = ensemble_var.sel(scenario=scenario)
        var_str += (
            f"{scenario}: {ensemble_var_scenario.mean(dim='model').values} "
            f"(model range: {ensemble_var_scenario.min(dim='model').values} "
            f"- {ensemble_var_scenario.max(dim='model').values})\n"
        )
    return var_str


ensemble_var_vals_temp = _get_ensemble_var_vals(ds_temp_yearly, data_var="temperature")
print("Variance of temperature due to internal variability:\n" + ensemble_var_vals_temp)

ensemble_var_vals_epi = _get_ensemble_var_vals(
    ds_days_suitable, data_var="portion_suitable"
)
print(
    "Variance of days suitable due to internal variability:\n" + ensemble_var_vals_epi
)

Variance of temperature due to internal variability:
Mean: 0.31012040566818777
ssp126: 0.3486231441872325 (model range: 0.24619719173094634 - 0.5763713788698618)
ssp370: 0.30074536914932404 (model range: 0.21592501973956302 - 0.40742815538571825)
ssp585: 0.2809927036680067 (model range: 0.21100558142558862 - 0.3680469815424778)

Variance of days suitable due to internal variability:
Mean: 197.8683444351872
ssp126: 240.47282873288867 (model range: 157.53794540398317 - 331.83127604815087)
ssp370: 182.4215480675421 (model range: 120.72187065880877 - 256.53808354873115)
ssp585: 170.71065650513086 (model range: 114.79369615314373 - 214.8445159812724)



In [5]:
import numpy as np

np.sqrt(
    ds_temp_yearly.climepi.variance_decomposition(
        internal_variability_method="polyfit", deg=3
    )
    .sel(source="internal")
    .isel(time=0)["temperature"]
    .compute()
)

<xarray.DataArray 'temperature' ()> Size: 8B
array(0.55688455)
Coordinates:
    lat       float64 8B 51.75
    lon       float64 8B -0.25
    location  <U11 44B 'London'
    time      int64 8B 2030
    source    <U8 32B 'internal'
Attributes:
    long_name:  Variance of temperature
    units:      °C²
    operation:  temporal_avg
    mode:       group_average
    freq:       year
    weighted:   True

In [6]:
# First years with 5% chance of exceeding example threshold of days suitable, with and
# without uncertainty due to internal variability.

days_suitable_threshold_example = 120


def _get_first_time_above_threshold(ds, threshold=None, data_var=None):
    return (
        ds["time"]
        .isel((ds[data_var] >= threshold).compute().argmax(dim=["time"]))
        .values.item()
    )


ds_days_suitable_95pc_with_icv = (
    ds_days_suitable.climepi.uncertainty_interval_decomposition(
        "portion_suitable", **{**ensemble_stats_kwargs, "uncertainty_level": 90}
    )
).sel(level="scenario_upper", drop=True)
first_year_95pc_with_icv = _get_first_time_above_threshold(
    ds_days_suitable_95pc_with_icv,
    threshold=days_suitable_threshold_example,
    data_var="portion_suitable",
)
print(
    f"First year with 5% of chance of above {days_suitable_threshold_example} "
    f"days suitable (with internal variability): {first_year_95pc_with_icv}"
)
ds_days_suitable_95pc_no_icv = (
    (
        ds_days_suitable.climepi.ensemble_stats(
            "portion_suitable", **ensemble_stats_kwargs
        )
    )
    .sel(stat="mean", drop=True)
    .quantile(0.95, dim=["scenario", "model"])
)
first_year_95pc_no_icv = _get_first_time_above_threshold(
    ds_days_suitable_95pc_no_icv,
    threshold=days_suitable_threshold_example,
    data_var="portion_suitable",
)
print(
    f"First year with 5% of chance of above {days_suitable_threshold_example} "
    f"days suitable (no internal variability): {first_year_95pc_no_icv}"
)


First year with 5% of chance of above 120 days suitable (with internal variability): 2038
First year with 5% of chance of above 120 days suitable (no internal variability): 2047
